# Challenge Analítica Avanzada e Inteligencia Artificial
## Centro de Investigación Coppel

### Elaborado por: Jhonathan Santacana

### Problema UNO

El objetivo de este problema es desarrollar un modelo de contactabilidad que asocie una probabilidad de contacto a cada cliente en función de ciertos segmentos horarios. Para resolver este problema, se utilizará el archivo 20210513_Challenge_AA.csv, que contiene información sociodemográfica y transaccional de los clientes.

## A análisis preliminar y proceso ETL:

1. Revisar Metadata: Revisa las características del archivo, como el formato, el tipo de datos, los tamaños y las columnas.

2. Cargar los datos del archivo .csv: Realiza la carga de los datos del archivo .csv en el entorno de trabajo.

3. Seleccionar horario, realizar el filtro y trabajar con un dataframe más pequeño: Selecciona un horario específico, aplica el filtro correspondiente y trabaja con un dataframe más reducido.

4. Indicar las dimensiones del nue, Indica el número de filas y columnas en el nuevo archivo, así como los nombres de las variables y una descripción general de los datos.

5. Obtener el número de observaciones, valores perdidos . valores NA.

6. Calcular estadí, icas: Calcula medidas estadísticas básicas como el promedio, los percentiles, las desviaciones estándar, los valores mínimos y máximos, y la mediana.

7. Elaborar un programa para crear un gráfico que proporcione una visión gene. al de los datos.

8. Elaborar un programa para crear un gráfico de barras que muestre la frecuencia de las categ y das der los datos: Diseña un programa que genere un gráfico de barras que muestre la frecuencia de las categorías de las variables continuas.

9. Realizar un análisis de corrrelaciones: Analiza las correlaciones entre las variables para identificar las que sean linealmente dependientes.

10. Establecer un criterio para determinar los factores influyentes con respeirto al objetivo: Define un criterio, como el valor de Information Value o el peso de la evidencia (Weight of Evidence), para determinar los factores influyentes en relación con la varirable objetivo. Enuméralos de mayor a menor según ses, no dudes en pedirlo.

In [1]:
# Librerias a utilizar, por defecto ya vienen instaladas en el entorno "pip install git jupyterlab numpy pandas plotly"
#! pip install pandasgui # para mejor visualizacion de los df
import os
from IPython.display import clear_output
import pandas as pd
from pandasgui import show
print("librerias cargadas correctamente") 

librerias cargadas correctamente


1. En la inspección visual del archivo, se detectaron separadores "|", y debido a su tamaño, se cargarán únicamente 5000 filas para revisar una parte del mismo. Esto nos permitirá visualizar rápidamente el número de columnas y los tipos de datos.

In [4]:
df = pd.read_csv("datos_internos/20210513_Challenge_AA.csv", sep="|", nrows=5000)

print(df.info())
show(df.head(10))

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ANIO             5000 non-null   int64  
 1   MES              5000 non-null   int64  
 2   CLIENTE          5000 non-null   int64  
 3   ESTADO           5000 non-null   object 
 4   INGRESO          4918 non-null   float64
 5   MORAS            5000 non-null   int64  
 6   SEXO             5000 non-null   object 
 7   ESTADOCIVIL      5000 non-null   object 
 8   FECHANACIMIENTO  5000 non-null   object 
 9   MARCACIONES      5000 non-null   int64  
 10  CONTACTOS        5000 non-null   int64  
 11  M1               5000 non-null   int64  
 12  C1               5000 non-null   int64  
 13  M2               5000 non-null   int64  
 14  C2               5000 non-null   int64  
 15  M3               5000 non-null   int64  
 16  C3               5000 non-null   int64  
 17  ANTIGUEDAD    

In [16]:
print(f"Los horarios para clasificacion son {list(df['HORARIO'].unique())}")

Los horarios para clasificacion son ['MEDIODIA', 'TARDE', 'NOCHE', 'MANANA']


2 y 3. Para manejar eficientemente grandes volúmenes de datos, los datos del archivo .csv se cargan por partes utilizando el iterador chunksize, lo que permite procesarlos de manera incremental. A continuación, se utiliza un ciclo for para iterar sobre los horarios y se aplica el filtro correspondiente a cada uno de ellos, creando así un dataframe más reducido para cada horario. Luego, estos dataframes se anexan a sus respectivos archivos .csv. Este enfoque facilita el análisis específico para cada horario en particular y permite trabajar con los datos de manera más eficiente.o.

In [2]:
def Creacion_csv_Horarios(file_path, chunk_size=100000, sep="|"):
    try:
        # visualizador de proceso
        en_proceso = "#"
        bytes_completo = os.path.getsize(file_path)
        bytes_avance = 0
        
        # Generar un iterador para leer el archivo CSV en partes, por defecto 1M de filas a la vez.
        reader = pd.read_csv(file_path, sep=sep, chunksize=chunk_size)
        
        encabezado = True
        horarios = ['MEDIODIA', 'TARDE', 'NOCHE', 'MANANA']
        for chunk in reader:
            # Visualizador de en proceso
            print(f"Avance estimado: {round(bytes_avance/bytes_completo,2)*100}%  {en_proceso}")
            bytes_avance = 0
            if len(en_proceso) >= 40:
                en_proceso = "#"
            else:
                en_proceso += "#"
                
            if encabezado:
                for Clasificador_Horario in horarios:
                    # Filtrar los datos según el horario
                    df_filtered = chunk.loc[chunk["HORARIO"] == Clasificador_Horario].drop("HORARIO", axis=1)
                    # Guardar cada parte en un archivo separado
                    df_filtered.to_csv(f"{file_path[:-4]}_{Clasificador_Horario}.csv", index=False)
                    bytes_avance += os.path.getsize(f"{file_path[:-4]}_{Clasificador_Horario}.csv")
                encabezado = False
            else:
                for Clasificador_Horario in horarios:
                    df_filtered = chunk.loc[chunk["HORARIO"] == Clasificador_Horario].drop("HORARIO", axis=1)
                    # Agregar los datos al archivo existente sin escribir los encabezados nuevamente
                    df_filtered.to_csv(f"{file_path[:-4]}_{Clasificador_Horario}.csv", mode='a', index=False, header=False) 
                    bytes_avance += os.path.getsize(f"{file_path[:-4]}_{Clasificador_Horario}.csv")
                    
            # Borrar la salida anterior /Visualizador de en proceso
            clear_output(wait=True)
                    
    except Exception as error:
        # Manejar el error y mostrar un mensaje informativo
        print("Ocurrió un error:", error)

# Leer el archivo .csv original y guardar el archivo por partes según los horarios
Creacion_csv_Horarios("datos_internos/20210513_Challenge_AA.csv")
print("creacion de archivos por horarios sin problemas")

creacion de archivos por horarios sin problemas


In [8]:

# Ruta del archivo
ruta_archivo1 = "datos_internos/20210513_Challenge_AA.csv"
ruta_archivo2 = "datos_internos/20210513_Challenge_AA.csv_MEDIODIA.csv"
# Obtener el tamaño del archivo en bytes
tamaño_bytes1 = os.path.getsize(ruta_archivo1)
tamaño_bytes2 = os.path.getsize(ruta_archivo2)

print(f"El archivo1 ocupa {tamaño_bytes1} bytes y el archivo2 {tamaño_bytes2}  bytes, es un {round(tamaño_bytes2/tamaño_bytes1,2)*100}%.")

El archivo1 ocupa 4008203183 bytes y el archivo2 1048178420  bytes, es un 26.0%.


10. Con base en los resultados de los puntos anteriores, elabora al menos cinco nuevas
variables que de acuerdo a los resultados estadísticos posean buen poder predictivo con
respecto a la variable objetivo. Explica la construcción de las variables.
11. Prepara un conjunto de entrenamiento y validación para realizar las determinaciones
correspondientes. Justifica el porcentaje usado para cada conjunto.
12. Elabora al menos tres propuestas de modelos de clasificación con las variables de tu
preferencia. Justifica las propuestas.
13. Sobre las propuestas realizadas, realiza la valoración de las métricas correspondientes
sobre las muestras de entrenamiento y validación. Selecciona un modelo ganador.
Justifica el uso de las métricas utilizadas y la selección del modelo que prefieres.
14. Explica cómo visualizas la implementación del modelo ganador, este podrá ser un punto
a considerar dentro de tu modelo preferido.
15. Prepara un informe ejecutivo que detalle cada uno de los puntos de esta rúbrica, desde
la carga hasta la presentación de resultados y su implementación. El entregable de este
challenge será un archivo con el código y el reporte ejecutivo en formato PDF.